In [6]:
# import packages
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, Input, Activation, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

import cv2

In [7]:
InputSamples = pd.read_csv('CapstoneMachineLearningTrainingInputs.csv')

Inputs = InputSamples.iloc[1:]
Outputs = InputSamples.iloc[0]

print(Inputs)

       0.0    1.0    2.0    3.0    4.0    5.0    6.0    7.0    8.0    9.0  \
1     59.0   14.0   80.0   75.0  107.0   65.0 -100.0  -28.0   -1.0  -46.0   
2     84.0  -20.0  107.0  116.0 -108.0  103.0  -60.0   12.0   34.0  -79.0   
3    108.0  -54.0 -120.0  -99.0  -67.0 -117.0  -20.0   51.0   72.0 -114.0   
4   -127.0  -87.0  -89.0  -58.0  -27.0  -82.0   21.0   89.0  110.0  105.0   
5   -107.0 -118.0  -57.0  -18.0   13.0  -48.0   62.0  126.0 -106.0   68.0   
..     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
96   -97.0  -48.0  -25.0 -127.0  -97.0  122.0  -30.0   24.0   96.0   97.0   
97   -62.0  -89.0   -9.0  -88.0  -56.0  -94.0    4.0   65.0  125.0   72.0   
98   -30.0 -128.0   10.0  -48.0  -16.0  -53.0   40.0  106.0 -101.0   45.0   
99     2.0   89.0   31.0   -7.0   25.0  -13.0   78.0 -109.0  -69.0   17.0   
100   31.0   51.0   54.0   34.0   66.0   26.0  116.0  -68.0  -36.0  -14.0   

     ...  1190.0  1191.0  1192.0  1193.0  1194.0  1195.0  1196.0  1197.0  \

In [19]:
def create_cnn(numSamples, numOutputs, filters=20, kernel_size=50, regress=False):
    
    model = Sequential([
        Conv1D(filters, kernel_size, activation='relu', input_shape=(numSamples,1)),
        Conv1D(filters, kernel_size, activation='relu'),
        Dropout(0.25),
        MaxPooling1D(pool_size=10),
        Flatten(),
        Dense(40, activation='relu'),
        Dense(numOutputs, activation='softmax')])
        
    # return the CNN
    return model

print(Inputs.shape[1]);
print(len(Outputs));

# Test split
X_train, X_test, y_train, y_test = train_test_split(Inputs.transpose(), Outputs.transpose(), test_size=0.25, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(Inputs, Outputs, test_size=0.25, random_state=42)

y_train = tf.keras.utils.to_categorical(y_train, 3)
y_test = tf.keras.utils.to_categorical(y_test, 3)

# Create cnn
model = create_cnn(X_train.shape[1], 3, 50, 30, True)
opt = Adam()
model.compile(loss="binary_crossentropy", optimizer=opt)

# Fit model
print(X_train.shape)
print("[INFO] training model...")
model.fit(x=X_train, y=y_train, 
    validation_data=(X_test,y_test),
    epochs=25, batch_size=10)

1200
1200
(900, 100)
[INFO] training model...
Epoch 1/25
90/90 [==============================] - 1s 4ms/step - loss: 0.7927 - val_loss: 0.0110
Epoch 2/25
90/90 [==============================] - 0s 3ms/step - loss: 0.0064 - val_loss: 5.3061e-04
Epoch 3/25
90/90 [==============================] - 0s 3ms/step - loss: 0.0236 - val_loss: 0.0032
Epoch 4/25
90/90 [==============================] - 0s 3ms/step - loss: 0.0062 - val_loss: 1.2415e-04
Epoch 5/25
90/90 [==============================] - 0s 3ms/step - loss: 0.0063 - val_loss: 0.0024
Epoch 6/25
90/90 [==============================] - 0s 3ms/step - loss: 0.0195 - val_loss: 0.0060
Epoch 7/25
90/90 [==============================] - 0s 4ms/step - loss: 0.0616 - val_loss: 0.1634
Epoch 8/25
90/90 [==============================] - 0s 4ms/step - loss: 0.0053 - val_loss: 8.3245e-05
Epoch 9/25
90/90 [==============================] - 0s 3ms/step - loss: 0.0018 - val_loss: 7.9142e-04
Epoch 10/25
90/90 [==============================] - 0s 

In [20]:
model.summary()
y_pred = model.predict(np.array(X_test));
y_pred = np.array(y_pred);
y_test = np.array(y_test);
rmse = np.sqrt(np.mean((y_pred - y_test)**2)); # ROOT MEAN SQUARE ERROR
mae = np.mean(abs(y_pred - y_test)); # MEAN ABSOLUTE ERROR
r2 = 1 - (np.sum((y_pred - y_test)**2))/((np.sum(np.mean(y_pred) - y_test))**2) # R2
print(y_pred);
print(y_test);
# plt.scatter(np.array(y_test)[:, 1]); # This a scatter of the test only
# plt.scatter(np.array(y_pred)[:, 0], np.array(y_pred)[:, 1]); # This adds the scatter of the predicted
print('RMSE:\n', rmse);
print('MAE:\n', mae);
print('R Squared:\n', r2);

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_18 (Conv1D)          (None, 71, 50)            1550      
                                                                 
 conv1d_19 (Conv1D)          (None, 42, 50)            75050     
                                                                 
 dropout_9 (Dropout)         (None, 42, 50)            0         
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 4, 50)            0         
 1D)                                                             
                                                                 
 flatten_9 (Flatten)         (None, 200)               0         
                                                                 
 dense_18 (Dense)            (None, 40)                8040      
                                                      

In [ ]:
print(np.array(y_test))


In [ ]:
print(y_pred.transpose())

In [ ]:
X_train.shape[1]